# Intro. to Snorkel: Extracting Spouse Relations from the News

## Part 4: Training our End Extraction Model

In this final section of the tutorial, we'll use the noisy training labels we generated in the last tutorial part to train our end extraction model.

For this tutorial, we will be training a simple - but fairly effective - logistic regression model.  More generally, however, Snorkel plugs in with many ML libraries including [TensorFlow](https://www.tensorflow.org/), making it easy to use almost any state-of-the-art model as the end extractor!

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import re
import numpy as np

# Connect to the database backend and initalize a Snorkel session
from lib.init import *
from snorkel.models import candidate_subclass
from snorkel.annotations import load_gold_labels

from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)

We repeat our definition of the `Spouse` `Candidate` subclass, and load the test set:

# 1 Training a `SparseLogReg` Discriminative Model
We use the training marginals to train a discriminative model that classifies each `Candidate` as a true or false mention. We'll use a random hyperparameter search, evaluated on the development set labels, to find the best hyperparameters for our model. To run a hyperparameter search, we need labels for a development set. If they aren't already available, we can manually create labels using the Viewer.

## Feature Extraction
Instead of using a deep learning approach to start, let's look at a standard sparse logistic regression model. First, we need to extract out features. This can take a while, but we only have to do it once!

In [2]:
from snorkel.annotations import FeatureAnnotator
featurizer = FeatureAnnotator()

In [3]:
F_train = featurizer.load_matrix(session, split=0)
F_dev = featurizer.load_matrix(session, split=1)
F_test = featurizer.load_matrix(session, split=2)

if F_train.size == 0:    
    %time F_train = featurizer.apply(split=0, parallelism=2)
if F_dev.size == 0:     
    %time F_dev  = featurizer.apply_existing(split=1, parallelism=2)
if F_test.size == 0:
    %time F_test = featurizer.apply_existing(split=2, parallelism=2)

Clearing existing...
Running UDF...
CPU times: user 13min 44s, sys: 1min 43s, total: 15min 28s
Wall time: 38min 34s
Clearing existing...
Running UDF...
CPU times: user 2min 27s, sys: 13.4 s, total: 2min 40s
Wall time: 5min 26s
Clearing existing...
Running UDF...
CPU times: user 2min 3s, sys: 11.4 s, total: 2min 15s
Wall time: 4min 55s




First, reload the training marginals:

In [4]:
from snorkel.annotations import load_marginals
train_marginals = load_marginals(session, F_train, split=0)

In [5]:
from snorkel.learning import SparseLogisticRegression
disc_model = SparseLogisticRegression()

The following code performs model selection by tuning our learning algorithm's hyperparamters.

In [ ]:
from snorkel.learning.utils import MentionScorer
from snorkel.learning import RandomSearch, ListParameter, RangeParameter

# Searching over learning rate
rate_param        = RangeParameter('lr', 1e-6, 1e-2, step=1, log_base=10)
l1_param          = RangeParameter('l1_penalty', 1e-6, 1e-2, step=1, log_base=10)
l2_param          = RangeParameter('l2_penalty', 1e-6, 1e-2, step=1, log_base=10)
batch_size_param  = ListParameter('batch_size', [32, 64, 128])
b_param           = ListParameter('b', [0.5, 0.6, 0.7])
balance_param     = ListParameter('rebalance', [0.0, 0.3, 0.5])

param_grid = [rate_param, l1_param, l2_param, batch_size_param, b_param, balance_param]

np.random.seed(1701)
searcher = RandomSearch(SparseLogisticRegression, param_grid, F_train,
                        Y_train=train_marginals, n=5, n_threads=1)

logreg, run_stats = searcher.fit(F_dev, L_gold_dev, n_epochs=2000, print_freq=250, n_threads=2)
print run_stats

Initialized RandomSearch search of size 5. Search space size = 3375.
Loading data...
Launching jobs...
[SparseLogisticRegression] Training model
[SparseLogisticRegression] n_train=8707  #epochs=2000  batch size=64
[SparseLogisticRegression] Training model
[SparseLogisticRegression] n_train=17311  #epochs=2000  batch size=32
[SparseLogisticRegression] Epoch 0 (1.35s)	Average loss=1.156878
[SparseLogisticRegression] Epoch 0 (3.10s)	Average loss=3.533144
[SparseLogisticRegression] Epoch 250 (425.35s)	Average loss=5.137200


In [ ]:
from snorkel.annotations import load_gold_labels

L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_dev.shape

In [ ]:
np.random.seed(1701)
searcher.fit(F_dev, L_gold_dev, n_epochs=50, rebalance=0.5, print_freq=25)

## Examining Features
Extracting features allows us to inspect and interperet our learned weights 

In [ ]:
w, _ = disc_model.get_weights()
largest_idxs = reversed(np.argsort(np.abs(w))[-50
                                              :])
for i in largest_idxs:
    print('Feature: {0: <70}Weight: {1:.6f}'.format(F_train.get_key(session, i).name, w[i]))

## Evaluate on Test Data

In [ ]:
from snorkel.annotations import load_gold_labels
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)

In [ ]:
_, _, _, _ = disc_model.score(session, F_test, L_gold_test)

# 2: Training an LSTM Discriminative Model
Deep learning allows us to train models without manually definining features

In [ ]:
train = session.query(Spouse).filter(Spouse.split == 0).order_by(Spouse.id).all()
dev = session.query(Spouse).filter(Spouse.split == 1).order_by(Spouse.id).all()
test = session.query(Spouse).filter(Spouse.split == 2).order_by(Spouse.id).all()

print('Training set:\t{0} candidates'.format(len(train)))
print('Dev set:\t{0} candidates'.format(len(dev)))
print('Test set:\t{0} candidates'.format(len(test)))

Load dev labels and convert to [0, 1] range

In [ ]:
dev_labels = (np.ravel(L_gold_dev.todense()) + 1) / 2

In [ ]:
from snorkel.contrib.rnn import reRNN

train_kwargs = {
    'lr':         0.01,
    'dim':        100,
    'n_epochs':   50,
    'dropout':    0.5,
    'rebalance':  0.25,
    'print_freq': 5,
    'batch_size': 32
}

lstm = reRNN(seed=1701, n_threads=None)
lstm.train(train, train_marginals, dev_candidates=dev,
           dev_labels=dev_labels, **train_kwargs)

## 3. Evaluating on the Test Set

In this last section of the tutorial, we'll get the score we've been after: the performance of the extraction model on the blind test set (`split` 2). First, we load the test set labels and gold candidates we made in Part III.

In [ ]:
from snorkel.annotations import load_gold_labels
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)

Now, we score using the discriminative model:

In [ ]:
_, _, _, _  = lstm.score(session, test, L_gold_test)